<a href="https://colab.research.google.com/github/Sherlin414/-customer-support-ticket-triage/blob/main/Week1_Internship_customerSupport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta


In [5]:
import pandas as pd

df = pd.read_csv("customer_support_tickets.csv")

print("File loaded successfully")
print("Rows:", df.shape[0], " | Columns:", df.shape[1])
df.head()


File loaded successfully
Rows: 8469  | Columns: 17


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [6]:
df.isnull().sum()


,0
Ticket ID,0
Customer Name,0
Customer Email,0
Customer Age,0
Customer Gender,0
Product Purchased,0
Date of Purchase,0
Ticket Type,0
Ticket Subject,0
Ticket Description,0


In [7]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df["clean_message"] = (
    df["Ticket Subject"].astype(str) + " " + df["Ticket Description"].astype(str)
).apply(clean_text)

df[["Ticket Subject","Ticket Description","clean_message"]].head()


,Ticket Subject,Ticket Description,clean_message
0,Product setup,I'm having an issue with the {product_purchase...,product setup i m having an issue with the pro...
1,Peripheral compatibility,I'm having an issue with the {product_purchase...,peripheral compatibility i m having an issue w...
2,Network problem,I'm facing a problem with my {product_purchase...,network problem i m facing a problem with my p...
3,Account access,I'm having an issue with the {product_purchase...,account access i m having an issue with the pr...
4,Data loss,I'm having an issue with the {product_purchase...,data loss i m having an issue with the product...


In [8]:
def classify_issue(text):

    rules = {
        "PAYMENT": ["payment", "billing", "invoice", "charge", "subscription"],
        "LOGIN": ["login", "password", "reset", "account access"],
        "DELIVERY": ["shipping", "delivery", "delay", "not received"],
        "REFUND": ["refund", "return", "money back"],
        "BUG": ["error", "bug", "crash", "issue", "fail"]
    }

    for label, keywords in rules.items():
        if any(word in text for word in keywords):
            return label

    return "GENERAL"

df["Issue_Type"] = df["clean_message"].apply(classify_issue)
df["Issue_Type"].value_counts()


,count
Issue_Type,
BUG,4916
LOGIN,1273
PAYMENT,962
REFUND,643
DELIVERY,540
GENERAL,135


In [9]:
def assign_priority(text):
    if any(word in text for word in ["urgent", "failed", "crash", "error", "not working"]):
        return "P0"
    if any(word in text for word in ["refund", "blocked", "payment issue"]):
        return "P1"
    if any(word in text for word in ["delay", "slow", "issue"]):
        return "P2"
    return "P3"

df["Priority_Level"] = df["clean_message"].apply(assign_priority)
df["Priority_Level"].value_counts()


,count
Priority_Level,
P2,5585
P0,1654
P1,1026
P3,204


In [10]:
sla_hours_map = {
    "P0": 4,
    "P1": 8,
    "P2": 24,
    "P3": 48
}

df["SLA_Hours"] = df["Priority_Level"].map(sla_hours_map)


In [11]:
df["Date of Purchase"] = pd.to_datetime(df["Date of Purchase"])

df["SLA_Due_Time"] = df["Date of Purchase"] + df["SLA_Hours"].apply(lambda h: timedelta(hours=h))

df[["Date of Purchase","Priority_Level","SLA_Hours","SLA_Due_Time"]].head()


,Date of Purchase,Priority_Level,SLA_Hours,SLA_Due_Time
0,2021-03-22,P2,24,2021-03-23
1,2021-05-22,P2,24,2021-05-23
2,2020-07-14,P3,48,2020-07-16
3,2020-11-13,P2,24,2020-11-14
4,2020-02-04,P2,24,2020-02-05


In [12]:
report = df.groupby(["Issue_Type","Priority_Level"]).size().reset_index(name="Ticket_Count")
report


,Issue_Type,Priority_Level,Ticket_Count
0,BUG,P0,823
1,BUG,P1,1
2,BUG,P2,4077
3,BUG,P3,15
4,DELIVERY,P0,88
5,DELIVERY,P1,21
6,DELIVERY,P2,415
7,DELIVERY,P3,16
8,GENERAL,P0,21
9,GENERAL,P3,114


In [14]:
output_path = "customer_support_triage_output.csv"
df.to_csv(output_path, index=False)

print("Exported successfully →", output_path)


Exported successfully → customer_support_triage_output.csv
